## CutQC Tutorial

In [1]:
# Import the CutQC module
from cutqc.main import CutQC
# We will use our provided function to generate example circuits. You can also create any circuits yourself.
from qiskit_helper_functions.non_ibmq_functions import generate_circ

### Generate and add circuits
The circuits need to be stored in a dictionary. The key is an arbitrary name for the circuit. Each item must contain
1. circuit: A quantum circuit
2. max_subcircuit_qubit: the max number of qubits each subcircuit can have
3. max_cuts: max number of cuts allowed for the cutter
4. num_subcircuits: list of number of subcircuits to try

In [2]:
circuits = {}
max_subcircuit_qubit = 2
for full_circ_size in [3]:
#     Replace circuit generation and circuit_name to your likes
    circuit = generate_circ(full_circ_size=full_circ_size, circuit_type='bv')
    circuit_name = 'bv_%d'%full_circ_size
    if circuit.num_qubits==0:
        continue
    else:
        circuits[circuit_name] = {
            'circuit':circuit,'max_subcircuit_qubit':max_subcircuit_qubit,'max_cuts':10,'num_subcircuits':[2,3,4,5]}

### Define hyperparameters
1. num_nodes: number of parallel compute nodes. As different computing clusters require very different setups, this functionality is currently not provided.
2. num_threads: number of parallel threads. Because of the parallel nature of Python codes, setting >1 threads may affect the readability of the runtime logs. The common-sense runtime tradeoffs are expected.
3. qubit_limit: set the max number of qubits your system RAM can handle. If qubit_limit<circuit size, CutQC runs in the DD mode.
4. eval_mode: 'sv' for statevector, 'qasm' for qasm simulator, 'runtime' for pseudo backend (for faster runtime benchmark purposes). Running on IBMQ support is coming soon.

In [3]:
num_nodes = 1
num_threads = 1
qubit_limit = 24
eval_mode = 'sv'

### Call CutQC
First, look for cuts and generate the necessary hyper information required for post-processing.

In [4]:
cutqc = CutQC(verbose=True) # verbose turns on detailed ouptut logs. Note that logs are long.
cutqc.cut(circuits=circuits) # Cut the circuits

******************** Cut ********************
     ┌───┐          ┌───┐     
q_0: ┤ H ├───────■──┤ H ├─────
     ├───┤       │  └───┘┌───┐
q_1: ┤ H ├───────┼────■──┤ H ├
     ├───┤┌───┐┌─┴─┐┌─┴─┐├───┤
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├┤ H ├
     └───┘└───┘└───┘└───┘└───┘
bv_3 on 2-q : 1 cuts -->
Subcircuit 0 : {'effective': 1, 'rho': 0, 'O': 1, 'd': 2}
Subcircuit 1 : {'effective': 2, 'rho': 1, 'O': 0, 'd': 2}
Estimated postprocessing cost = 3.200e+01
--> bv_3 subcircuit_instances:
subcircuit_0_instance_idx      #shots     init                           meas                          
0                              8192       ('zero', 'zero')               ('comp', 'I')                 
     ┌───┐          ┌───┐
q_0: ┤ H ├───────■──┤ H ├
     ├───┤┌───┐┌─┴─┐└───┘
q_1: ┤ X ├┤ H ├┤ X ├─────
     └───┘└───┘└───┘     
1                              0          ('zero', 'zero')               ('comp', 'Z')                 
     ┌───┐          ┌───┐
q_0: ┤ H ├───────■──┤ H ├
     ├───┤┌───┐┌─┴─┐└───┘
q_1

### Evaluate the circuits based on the cuts found

In [5]:
reconstructed_probs = cutqc.evaluate(circuits=circuits,
                                     eval_mode=eval_mode,
                                     qubit_limit=qubit_limit,
                                     num_nodes=num_nodes,
                                     num_threads=num_threads,
                                     ibmq=None)
print(reconstructed_probs)

******************** evaluation mode = sv ********************
--> Running Subcircuits
7 total
{('bv_3', 0, ('zero', 'zero'), ('comp', 'I')): array([0., 1.]), ('bv_3', 0, ('zero', 'zero'), ('comp', 'Z')): array([0., 0.])}
{('bv_3', 1, ('zero', 'zero'), ('comp', 'comp')): array([0.5, 0. , 0. , 0.5])}
{('bv_3', 1, ('zero', 'one'), ('comp', 'comp')): array([0.5, 0. , 0. , 0.5])}
{('bv_3', 0, ('zero', 'zero'), ('comp', 'X')): array([ 0., -1.])}
{('bv_3', 1, ('zero', 'plus'), ('comp', 'comp')): array([1., 0., 0., 0.])}
{('bv_3', 0, ('zero', 'zero'), ('comp', 'Y')): array([0., 0.])}
{('bv_3', 1, ('zero', 'plusI'), ('comp', 'comp')): array([0.5, 0. , 0. , 0.5])}
--> Attribute shots
circuit_name    subcircuit_idx  subcircuit_instance_idx   coefficient, subcircuit_entry_idx
bv_3            0               0                         [(1, 0)]                      
bv_3            0               1                         [(1, 3)]                      
bv_3            1               0             

The final output of the reconstructed_probs are in a scrambled order due to the placement of the subcircuits. It is trivial to know the order of each state adhering to the original qubit order. However, converting all quantum states to the original order still takes 2^n. In practise, you can look at the output, figure out which states you are interested in, and just earsily convert those. Here we verify for every state that the CutQC output is the same as the original circuit.

Note that if the input circuit is large, `verify` may take a long time to run.

In [6]:
errors = cutqc.verify(circuits=circuits,
                      num_nodes=num_nodes,
                      num_threads=num_threads,
                      qubit_limit=qubit_limit,
                      eval_mode=eval_mode)

******************** Verify ********************
Circuit Name         QPU        Error                         
bv_3                 sv         5.6e-32                       



Since we are using statevector simulator as the QPU backend, the error is 0 within numerical inaccuracies.